In [ ]:
# this workboook illustrates how to extract data from the JSON file returned by videointelligence
# the example transcript was generated from the industry archives file:
# YOUTH SMOKING PREVENTION 7 SPOTS
# https://archive.org/details/tobacco_wbr62a00

In [3]:
import pandas as pd
import glob
import json
import numpy as np
from pandasql import sqldf

In [4]:
pysqldf = lambda q: sqldf(q, globals())

In [5]:
files = glob.glob('sample_json/*.json', recursive=True)

In [ ]:
#len(files)

In [ ]:
#files[0]

In [ ]:
#f = open(files[0])
#data = json.load(f) 

In [ ]:
#data.keys()

In [ ]:
#data['annotation_results'][0].keys()

In [ ]:
#data['annotation_results'][1].keys()

In [ ]:
#data['annotation_results'][0]['input_uri']

In [ ]:
#data['annotation_results'][0]['segment_label_annotations'][0]['entity']['description']

In [ ]:
#data['annotation_results'][0]['shot_label_annotations'][0]['entity']['description']

In [ ]:
#data['annotation_results'][0]['text_annotations'][0]['text']

In [ ]:
#data['annotation_results'][1]['speech_transcriptions'][2]['alternatives'][0]['transcript']

In [ ]:
#data['annotation_results'][1]['speech_transcriptions'][2]['alternatives'][0]['confidence']

In [6]:
label_annotations = []
text_annotations = []
logo_annotations = []
speech_annotations = []

for file in files:
    
    f = open(file)
    data = json.load(f) 
    
    for i in range(len(data['annotation_results'])):
    
        ar = data['annotation_results'][i]

        uri = ar['input_uri']

        if 'shot_label_annotations' in ar:

            slas = []
            for sla in ar['shot_label_annotations']:
                label = sla['entity']['description']
                slas.append(label)

            label_annotations.append((uri, ','.join(slas)))

        if 'text_annotations' in ar:
            tas = []
            for ta in ar['text_annotations']:
                text = ta['text']
                tas.append(text)

            text_annotations.append((uri, ' '.join(tas)))

        if 'logo_recognition_annotations' in ar.keys():                         
            lras = []
            for lra in ar['logo_recognition_annotations']:                  
                logo = lra['entity']['description']
                lras.append(logo)

            logo_annotations.append((uri, ','.join(lras)))


        #['speech_transcriptions'][2]['alternatives'][0]['transcript']
        if 'speech_transcriptions' in ar:

            sts = []
            scs = []

            for st in ar['speech_transcriptions']:

                alt_conf = st['alternatives'][0]

                if 'transcript' in alt_conf:
                    speech = alt_conf['transcript']
                    speech_confidence = alt_conf['confidence']
                    #print(speech_confidence)
                    sts.append(speech)
                    if speech_confidence == []:
                        speech_confidence = 0
                        
                    scs.append(float(speech_confidence))

            speech_annotations.append((uri, ''.join(sts), np.mean(scs)))
        
    #print(file)

In [7]:
df_label_annotations = pd.DataFrame(label_annotations, columns=['uri', 'label'])
df_text_annotations = pd.DataFrame(text_annotations, columns=['uri', 'text'])
df_logo_annotations = pd.DataFrame(logo_annotations, columns=['uri', 'logo'])
df_speech_annotations = pd.DataFrame(speech_annotations, columns=['uri', 'transcript', 'confidence'])

In [8]:
df_annotations = pysqldf("""
SELECT 
    sa.uri,
    sa.transcript,
    sa.confidence,
    ta.text,
    la.label,
    lga.logo
FROM
    df_speech_annotations sa
LEFT JOIN
    df_label_annotations la
ON
    sa.uri = la.uri
LEFT JOIN
    df_text_annotations ta
ON
    sa.uri = ta.uri
LEFT JOIN
    df_logo_annotations lga
ON
    sa.uri = lga.uri
""")

In [9]:
df_annotations.insert(0, 'file_name', df_annotations['uri'].str.split('/').str[-1][:-4])

In [10]:
df_annotations.insert(0, 'identifier', df_annotations['uri'].str.split('/').str[-2])

In [11]:
df_annotations.head(2)

,identifier,file_name,uri,transcript,confidence,text,label,logo
0,input-idl-video-files,NaN,/input-idl-video-files/tobacco_demo.mp4,I think smoking makes you look cool. No way. W...,0.825757,P MErtis USA SUOLON GENERAL'S WARNING: Smoking...,"black hair,finger,motor vehicle,sports,car,can...","Think Mutual Bank,Isuzu Philippines,New York J..."


In [13]:
df_annotations['text_content'] = df_annotations['transcript']

In [14]:
df_annotations.to_csv('sample_json/sample_json.csv', index=False)